In [3]:
import pandas as pd
import numpy as np
import spacy
import json
import jsonlines
nlp = spacy.load("nb_core_news_sm")

In [4]:
ton = pd.read_csv('./data/ton.csv', low_memory=False)

In [5]:
df_text = ton[['text']].dropna().sample(frac=1).reset_index(drop=True)

In [6]:
def get_labels():
    df_party = ton[['party_name']].dropna()
    df_labels = df_party.party_name.unique()
    return(df_labels.tolist())

In [25]:
def to_spacy_format(text, start, end):
    entity_list = [(start, end, 'POLITICAL PARTY')]
    entity_dict = {'entities': entity_list}
    return(text, entity_dict)

In [26]:

label_list = get_labels()
annotation_list = []
df_test_ner = df_text[:1000]
for index, row in df_test_ner.iterrows():
    doc_sents = nlp(row['text']).sents
    for sent in doc_sents:
        sentence = (str(sent))
        doc = nlp(sentence)
        for ent in doc.ents:
            if len(annotation_list) < 1000:
                if ent.text in label_list:
                    to_print = to_spacy_format(doc.text, ent.start_char, ent.end_char)
                    annotation_list.append(to_print)
print(annotation_list)

        

[('Ønsker representanten Jensen å ta opp de forslagene som Fremskrittspartiet står sammen med andre om?', {'entities': [(56, 74, 'POLITICAL PARTY')]}), ('Høyre er svært opptatt av norsk næringslivs konkurranseevne i det internasjonale markedet.', {'entities': [(0, 5, 'POLITICAL PARTY')]}), ('Høyre konstaterer også i dag at regjeringen viser liten vilje og evne til å ta tak i disse utfordringene.', {'entities': [(0, 5, 'POLITICAL PARTY')]}), ('På denne bakgrunn ser vi det i Høyre som lite relevant å ilegge en ytterligere stabilitetsavgift med formål å dekke inn statens kostnader.', {'entities': [(31, 36, 'POLITICAL PARTY')]}), ('Høyre vil likevel understreke at dette unntaket stort sett gjelder for hele næringen globalt sett, og vil advare regjeringen mot å innføre norske særavgifter som svekker norsk finansnærings konkurranseevne.', {'entities': [(0, 5, 'POLITICAL PARTY')]}), ('Høyre er positive til det arbeidet som nå gjøres gjennom Baselkomiteen og i EU med nye krav til kapitaldeknin

In [31]:
with jsonlines.open('training_data.jsonl', 'w') as writer:
    for line in annotation_list:
        text = json.dumps(line)
        text = json.loads(text)
        writer.write(text)